In [11]:
!pip install ortools

     |████████████████████████████████| 11.8 MB 38 kB/s eta 0:00:011


In [1]:
import pandas as pd
data = pd.read_csv('family_data.csv',index_col='family_id')
data

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...
4995,16,1,66,33,18,70,56,46,86,60,4
4996,88,66,20,17,26,54,81,91,59,48,2
4997,32,66,54,17,27,21,74,81,3,7,6


In [2]:
# n代表家庭成员个数，如果满足第choice个需求，需要的penalty(惩罚)
def get_penalty(n, choice):
    penalty = None
    if choice == 0:
        penalty = 0
    if choice == 1:
        penalty = 50
    if choice == 2:
        penalty = 50 + 9 * n
    if choice == 3:
        penalty = 100 + 9 * n
    if choice == 4:
        penalty = 200 + 9 * n
    if choice == 5:
        penalty = 200 + 18 * n
    if choice == 6:
        penalty = 300 + 18 * n
    if choice == 7:
        penalty = 300 + 36 * n
    if choice == 8:
        penalty = 400 + 36 * n
    if choice == 9:
        penalty = 500 + (36 + 199) * n
    if choice > 9:
        penalty = 500 + (36 + 398) * n
    return penalty

In [3]:
import numpy as np 
N_DAYS = 100 # 安排的天数
N_FAMILY = 5000 #家庭ID的个数
MIN_OCCUPANCY = 125 # 最小承载量
MAX_OCCUPANCY = 300 # 最大承载量
# 计算pcost_mat，每个家庭在什么时候（day0-99）访问时的penalty
# 大小5000*100的矩阵
pcost_mat = np.full(shape=(N_FAMILY,100),fill_value=999999)
for f in range(N_FAMILY):
    # 家庭成员数
    f_num = data.loc[f,'n_people']
    # 对于第f个家庭，初始化pcost = other choice下的penalty
    pcost_mat[f,:] = get_penalty(f_num,10) #初始值最大惩罚
    # 计算choice 0-9 的penalty
    for choice in range(10):# choice 0-9
        temp = data.loc[f][choice] #choice的天数
        penalty = get_penalty(f_num,choice) # 得到对应choice的惩罚
        pcost_mat[f,temp-1] = penalty # 因为下标是从0开始，所以要在天数基础上-1才是下标值
pcost_mat

array([[2236, 2236, 2236, ..., 2236, 2236, 2236],
       [2236, 2236, 2236, ..., 2236, 2236, 2236],
       [1802, 1802, 1802, ..., 1802, 1802,    0],
       ...,
       [3104, 3104,  616, ..., 3104, 3104, 3104],
       [ 390, 2670, 2670, ..., 2670, 2670, 2670],
       [2236, 2236, 2236, ..., 2236, 2236, 2236]])

In [4]:
pcost_mat.shape

(5000, 100)

In [5]:
# 计算accounting penalty矩阵，前一天的参观人数和当天的参观人数
acost_mat = np.zeros(shape=(MAX_OCCUPANCY+1,MAX_OCCUPANCY+1),dtype=np.float64)
for i in range(acost_mat.shape[0]):# 当天安排的人数
    for j in range(acost_mat.shape[1]):# 前一天安排的人数
        diff = abs(i-j)
        acost_mat[i,j] = max(0,(i - 125) / 400 * i ** (0.5 + diff/50.0))
acost_mat

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.16316072e+15, 3.71482922e+15, 3.31477861e+15, ...,
        7.46610759e+00, 8.36716954e+00, 9.37697794e+00],
       [4.79555148e+15, 4.27883100e+15, 3.81778713e+15, ...,
        8.43020770e+00, 7.52185316e+00, 8.43020770e+00],
       [5.52415954e+15, 4.92860244e+15, 4.39725208e+15, ...,
        9.51970597e+00, 8.49339085e+00, 7.57772228e+00]])

In [6]:
FAMILY_SIZE = data['n_people'].values
FAMILY_SIZE

array([4, 4, 3, ..., 6, 5, 4])

In [7]:
DESIRED = data.values[:,:-1]-1
print(len(DESIRED))
DESIRED

5000


array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]])

In [8]:
from ortools.linear_solver import pywraplp

# 线性规划
def solveLP():
    solver= pywraplp.Solver('AssignmentProblem', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    x = {} #family_id在第j天是否参观
    candidates = [[] for x in range(N_DAYS)] # 定义了len为100的list
    for i in range(N_FAMILY):#family_id
        for j in DESIRED[i,:]:# family_id的choice
            candidates[j].append(i) # 在第j天，有第i个family参观
            x[i,j] = solver.BoolVar('x[%i,%i]' %(i,j)) # x[%i,%i]中的i代表integer类型
    
    daily_occupancy = [solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]])\
                        for j in range(N_DAYS)] # j代表1-100天
    family_presence = [solver.Sum(x[i,j]  for j in DESIRED[i,:])\
                        for i in range(N_FAMILY)]
    preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] for i in range(N_FAMILY)\
                                for j in DESIRED[i,:]])
    
    # 满足preference_cost最小
    solver.Minimize(preference_cost)
    
    # 人为增加约束条件为了更加拟合 Constraints
    for j in range(N_DAYS-1):
        # 当前人数不超过前一天人数+25
        solver.Add(daily_occupancy[j] - daily_occupancy[j+1]<=25)
        solver.Add(daily_occupancy[j+1] - daily_occupancy[j]<=25)
        
    # 每个家庭都在10个choice中出现一次
    for i in range(N_FAMILY):
        solver.Add(family_presence[i]==1)
    # 每天访问人数约束
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j]>=MIN_OCCUPANCY)
        solver.Add(daily_occupancy[j]<=MAX_OCCUPANCY)
    result = solver.Solve()
    
    temp = [(i,j,x[i,j].solution_value()) for i in range(N_FAMILY)\
             for j in DESIRED[i,:] if x[i,j].solution_value()>0]
    # 得到参观日期安排
    df = pd.DataFrame(temp,columns=['family_id','day','result'])
    return df

In [9]:
result = solveLP()
result

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5077,4995,15,1.0
5078,4996,87,1.0
5079,4997,31,1.0
5080,4998,91,1.0


In [10]:
# 设置阈值
THRS = 0.999
assigned_df = result[result.result > THRS]
assigned_df

,family_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5077,4995,15,1.0
5078,4996,87,1.0
5079,4997,31,1.0
5080,4998,91,1.0


In [11]:
# 没有安排好的，即不为0和1
unassigned_df = result[(result.result < THRS) & (result.result > 1-THRS)]
unassigned_df

,family_id,day,result
60,59,38,0.2500
61,59,14,0.7500
242,240,32,0.7500
243,240,56,0.2500
360,357,24,0.4625
...,...,...,...
4992,4912,8,0.4000
4994,4914,38,0.6000
4995,4914,43,0.4000
5042,4961,53,0.7500


In [12]:
# 计算每天访问的人数（根据assigned_df）
assigned_df['family_size'] = FAMILY_SIZE[assigned_df.family_id]
assigned_df

<ipython-input-12-240f183a50ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assigned_df['family_size'] = FAMILY_SIZE[assigned_df.family_id]


,family_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5077,4995,15,1.0,4
5078,4996,87,1.0,2
5079,4997,31,1.0,6
5080,4998,91,1.0,5


In [13]:
# 按照day进行聚合
occupancy = assigned_df.groupby('day').family_size.sum().values
occupancy

array([289, 271, 294, 293, 263, 242, 223, 247, 273, 297, 288, 292, 275,
       250, 245, 272, 292, 292, 271, 248, 223, 244, 264, 291, 292, 296,
       273, 249, 234, 251, 278, 283, 252, 235, 205, 184, 202, 233, 253,
       226, 210, 183, 204, 229, 247, 281, 256, 223, 204, 198, 222, 248,
       256, 223, 208, 185, 173, 196, 219, 198, 174, 141, 124, 121, 149,
       170, 178, 156, 136, 123, 125, 135, 158, 185, 158, 134, 121, 125,
       142, 167, 186, 167, 138, 120, 128, 155, 174, 203, 177, 152, 130,
       122, 130, 158, 183, 158, 128, 125, 122, 124])

In [14]:
min_occupancy = np.array([max(0,MIN_OCCUPANCY - x) for x in occupancy])
min_occupancy

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 4, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 1])

In [15]:
max_occupancy = np.array([MAX_OCCUPANCY - x for x in occupancy])
max_occupancy

array([ 11,  29,   6,   7,  37,  58,  77,  53,  27,   3,  12,   8,  25,
        50,  55,  28,   8,   8,  29,  52,  77,  56,  36,   9,   8,   4,
        27,  51,  66,  49,  22,  17,  48,  65,  95, 116,  98,  67,  47,
        74,  90, 117,  96,  71,  53,  19,  44,  77,  96, 102,  78,  52,
        44,  77,  92, 115, 127, 104,  81, 102, 126, 159, 176, 179, 151,
       130, 122, 144, 164, 177, 175, 165, 142, 115, 142, 166, 179, 175,
       158, 133, 114, 133, 162, 180, 172, 145, 126,  97, 123, 148, 170,
       178, 170, 142, 117, 142, 172, 175, 178, 176])

In [16]:
#进行 使用整数规划求解
def solveIP(families,min_occupancy,max_occupancy):
    # 创建求解器
    solver = pywraplp.Solver('AssignmentProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    # 需要安排的家庭
    n_families = len(families)
    x = {} # family_id在第j天是否参观
    candidates = [[] for x in range(N_DAYS)] # 定义了len为100的list
    for i in families:#family_id
        for j in DESIRED[i,:]:# family_id的choice
            #print(j)
            candidates[j].append(i) # 在第j天，有第i个family参观
            x[i,j] = solver.BoolVar('x[%i,%i]' %(i,j)) # x[%i,%i]中的i代表integer类型
  
    daily_occupancy = [solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]])\
                        for j in range(N_DAYS)] # j代表1-100天
    
    family_presence = [solver.Sum(x[i,j]  for j in DESIRED[i,:])\
                        for i in families]

    preference_cost = solver.Sum([pcost_mat[i,j] * x[i,j] for i in families\
                                for j in DESIRED[i,:]])
    
    # 满足preference_cost最小
    solver.Minimize(preference_cost)    
    
    # 每个家庭都在10个choice中出现一次
    for i in range(n_families):
        solver.Add(family_presence[i]==1)
    # 每天访问人数约束
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j]>=min_occupancy[j])
        solver.Add(daily_occupancy[j]<=max_occupancy[j])
    result = solver.Solve()
    
    temp = [(i,j) for i in families\
             for j in DESIRED[i,:] if x[i,j].solution_value()>0]
    # 计算剩余家庭的安排
    df = pd.DataFrame(temp,columns=['family_id','day'])
    return df

In [17]:
# 没有安排的family_id
unassigned = unassigned_df.family_id.unique()
result = solveIP(unassigned,min_occupancy,max_occupancy)
unassigned

array([  59,  240,  357,  488,  500,  724,  892,  926,  941,  961,  965,
       1012, 1106, 1176, 1218, 1235, 1287, 1387, 1444, 1484, 1536, 1610,
       1653, 1720, 1905, 1933, 1977, 2004, 2101, 2135, 2195, 2229, 2318,
       2344, 2353, 2395, 2641, 2778, 2862, 3033, 3054, 3104, 3118, 3228,
       3278, 3280, 3348, 3470, 3504, 3642, 3649, 3786, 4045, 4077, 4084,
       4404, 4421, 4431, 4471, 4518, 4619, 4628, 4761, 4820, 4850, 4886,
       4912, 4914, 4961])

In [18]:
# 合并数据
df_concat = pd.concat((assigned_df[['family_id','day']],result)).sort_values('family_id')
df_concat

,family_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
5077,4995,15
5078,4996,87
5079,4997,31
5080,4998,91


In [19]:
from numba import njit
@njit(fastmath=True)
def pcost(prediction):
    daily_occupancy = np.zeros(N_DAYS+1,dtype=np.int64)
    penalty = 0
    for (i,p) in enumerate(prediction):
        n = FAMILY_SIZE[i]
        penalty += pcost_mat[i,p]
        daily_occupancy[p] += n
    return penalty,daily_occupancy

@njit(fastmath=True)
def acost(daily_occupancy):
    accounting_cost = 0
    num_out_of_range = 0
    for day in range(N_DAYS):
        n_p1 = daily_occupancy[day+1] # 前一天
        n = daily_occupancy[day] # 当天
        num_out_of_range += (n>MAX_OCCUPANCY) or (n<MIN_OCCUPANCY)
        accounting_cost += acost_mat[n,n_p1]
    return accounting_cost,num_out_of_range

@njit(fastmath=True)
def cost_function(prediction):
    penalty,daily_occupancy = pcost(prediction) #统计preference cost和每天承载数量
    accounting_cost,num_out_of_range = acost(daily_occupancy) # 根据每天承载数量计算accounting cost
    final_score = penalty + accounting_cost + num_out_of_range * 9999999
    return final_score

In [20]:
prediction = df_concat.day.values
print(cost_function(prediction))

109202.84525824538


In [21]:
def save_result(pred,filename):
    result = pd.DataFrame(range(N_FAMILY),columns=['family_id'])
    result['assigned_day'] = pred + 1
    result.to_csv(filename,index=False)
    print(filename + ' saved')
    return result
result = save_result(prediction,'submission.csv')

submission.csv saved


In [22]:
# 寻找更好的替代方案
def find_better(pred):
    fobs = np.argsort(FAMILY_SIZE) # 返回数组从小到大的索引
    #print(np.sort(FAMILY_SIZE)) # 对FAMILY_SIZE按从小到大的顺序排序
    score = cost_function(pred)
    original_score = np.inf #打擂法 正无穷初始值
    
    # 如果找不到更新则退出
    while score < original_score:
        original_score = score
        for family_id in fobs:
            for pick in range(10):
                # 得到family_id在choice pick的日期day
                day = DESIRED[family_id, pick]
                # 该family的原有日期oldvalue
                oldvalue = pred[family_id]
                # 将原有oldvalue替换为现在的choice pick
                pred[family_id] = day
                # 重新计算分数
                new_score = cost_function(pred)
                # 如果比原来分数小，更新choice成功
                if new_score < score:
                    score = new_score
                else:# 设置为原来的oldvalue
                    pred[family_id] = oldvalue
        print(score,end='\r')
    print(score)

In [ ]:
find_better(prediction)